In [2]:
import cobra.test
import cobra.io
import pandas as pd

In [3]:
%%capture
# ignore output because it is too much
# "Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged...use fbc:fluxObjective instead:"
iMR=cobra.io.read_sbml_model("iMR539.xml")

In [4]:
display(iMR)

Name,M_maripaludis_probanno_xml
Memory address,0x07f13b181a9d0
Number of metabolites,710
Number of reactions,688
Number of groups,137
Objective expression,1.0*biomass0 - 1.0*biomass0_reverse_9c18d
Compartments,"Cytosol, Extracellular"


In [24]:
# output medium
medium_list = list(iMR.medium.keys())
# medium_list[-2] EX_Flagellin_LSQBKT_e0_RSQBKT_
del medium_list[-2]
def getMediumContent(model):
    df=pd.DataFrame()
    for i in medium_list:
        try:
            compound=model.metabolites.get_by_id(i[3:11]+"_c0")
        except KeyError:
            compound=model.metabolites.get_by_id(i[3:11]+"_e0")
        df[i]=pd.Series({'Metabolite identifier': compound.id, 'Name':compound.name, \
            'Formula':compound.formula}) 
    return df
df=getMediumContent(iMR)
compound=iMR.metabolites.get_by_id("FLGN_e0")
df["FLGN_e0"]=pd.Series({'Metabolite identifier': compound.id, 'Name':compound.name, \
            'Formula':compound.formula})
df.T.to_csv("iMR.medium.csv")

In [29]:
# output solution with detail
def getSolutionDetail(model):
    solution=model.optimize().to_frame()
    df=pd.DataFrame()
    for i in solution.index:
        reaction=model.reactions.get_by_id(i)
        df[i]=pd.Series({'Reaction identifier': reaction.id, \
            'Stoichiometry':reaction.build_reaction_string(use_metabolite_names=True), \
            'GPR':reaction.gene_name_reaction_rule}) 
    return solution.join(df.T)
getSolutionDetail(iMR).to_csv("iMR.FBAsolution.csv")

In [53]:
# output exchange_fluxes summary
summary=iMR.summary().to_frame()
summary.to_csv("iMR.exchange_fluxes.csv")
# output exchange_fluxes summary detail
id=list(summary.iloc[:, 0])+list(summary.iloc[:2, 2])
df=pd.DataFrame()
for i in id:
    compound=iMR.metabolites.get_by_id(i)
    df[i]=pd.Series({'Metabolite identifier': compound.id, 'Name':compound.name, \
        'Formula':compound.formula}) 
df.T.to_csv("iMR.exchange_fluxes.detail.csv")

In [54]:
summary


IN_FLUXES               OUT_FLUXES            OBJECTIVES          
            ID        FLUX           ID       FLUX         ID      FLUX
0  cpd11640_e0  190.527078  cpd00001_e0  93.716251   biomass0  0.097315
1  cpd00011_e0   45.257937  cpd01024_e0  50.000000        NaN       NaN
2  cpd00029_e0    3.666756          NaN        NaN        NaN       NaN

In [41]:
summary

IN_FLUXES               OUT_FLUXES            OBJECTIVES          
            ID        FLUX           ID       FLUX         ID      FLUX
0  cpd11640_e0  190.527078  cpd00001_e0  93.716251   biomass0  0.097315
1  cpd00011_e0   45.257937  cpd01024_e0  50.000000        NaN       NaN
2  cpd00029_e0    3.666756          NaN        NaN        NaN       NaN

In [48]:
list(summary.iloc[:2, 2])

['cpd00001_e0', 'cpd01024_e0']